### Detour

In [ ]:
import pandas as pd

df = pd.read_csv("detour/Detour_data_new.csv")
df_new = pd.DataFrame(columns = ["Video","Box","Passes"])
print("Vids to do:", len(pd.unique(df.Video)))

vid_count = 0
for vid in pd.unique(df.Video):
    vid_count += 1
    print("Vid:", vid_count)
    temp_df = df[df.Video == vid]
    for box in pd.unique(temp_df.Box):
        box_df = temp_df[temp_df.Box == box]
        box_df.reset_index(drop=True, inplace=True)
        count = 0
        pos = "No"
        for index, row in box_df.iterrows():
            if row.Past_barrier != pos:
                count += 1
                pos = row.Past_barrier
        short_df = pd.DataFrame({"Video": [vid], "Box": [box], "Passes": count})
        df_new = pd.concat([df_new, short_df], ignore_index=True)

df_new.to_csv("detour/Sum_detour.csv", index=False)

### Open field and mirror

In [ ]:
import pandas as pd
import math
import os

df = pd.read_csv("OF_mirror/OF_mirror_data_new.csv", keep_default_na=False)
final_df = pd.DataFrame(columns = ["Video","Vid_frames","Vid_width","fps","Box","Detected_frames",
                                   "Door_open","First_frame","Full_emergence","Emerged",
                                   "Distance","Shorter","Detected_frames_OF","Longest_no_detect_OF",
                                   "Frames_mirror","Time_mirror","Detected_frames_mir",
                                   "Frames_red_emerge_pt","Time_red_emerge_pt","Frames_red_OF_pt","Time_red_OF_pt",
                                   "Frames_red_emerge_box","Time_red_emerge_box","Frames_red_OF_box","Time_red_OF_box",
                                   "Frames_blue_emerge_pt","Time_blue_emerge_pt","Frames_blue_OF_pt","Time_blue_OF_pt",
                                   "Frames_blue_emerge_box","Time_blue_emerge_box","Frames_blue_OF_box","Time_blue_OF_box"])

timing_df = pd.read_csv("OF_mirror/Timing_OF_mir.csv",keep_default_na=False)
timing_list = pd.unique(timing_df.Video)

if os.path.exists("OF_mirror/Sum_OF_mir.csv"):
    df_summed = pd.read_csv("OF_mirror/Sum_OF_mir.csv", keep_default_na=False)
    summed_vids = pd.unique(df_summed.Video)
else:
    summed_vids = []

def dist_func(pt1, pt2):
    return(math.hypot(pt2[0]-pt1[0],pt2[1]-pt1[1]))

count = 0
for vid in pd.unique(df.Video):
    count += 1
    print(count, vid)
    if vid not in timing_list:
        print("No timing data")
        continue
    if vid in summed_vids:
        print("Already summed")
        continue
    temp_df = df[df.Video == vid]
    vid_frames = list(pd.unique(temp_df.Frames))[0]
    width = list(pd.unique(temp_df.Width))[0]
    if width == 1920:
        pix_to_cm = 420/57
    if width == 2560:
        pix_to_cm = 747/57
    
    l = [idx for idx, item in enumerate(vid.lower()) if "_" in item]
    start_pos = l[0] + 1
    end_pos = l[1]
    pos_name = vid[start_pos:end_pos]
    sum_df = pd.DataFrame(columns = final_df.columns)
    
    door_open = list(pd.unique(timing_df.Door_open[timing_df.Video == vid]))[0]
    box_open = list(pd.unique(timing_df.Box_opened[timing_df.Video == vid]))[0]
    mirror_drop = list(pd.unique(timing_df.Mirror_drop[timing_df.Video == vid]))[0]
    
    lids = False
    if len(pd.unique(temp_df.Lid_pt)) == 1:
        if pd.unique(temp_df.Lid_pt) != "NA":
            lids = True
    else:
        lids = True
    
    fps = 3  
    
    frames_20 = int(1200 * fps)
    frames_15 = int(900*fps)
    
    for box in pd.unique(temp_df.Box):
        box_df = temp_df[temp_df.Box == box]
        if pos_name == "B14":
            if box == 1:
                box_nr = 4
            elif box == 2:
                box_nr = 2
            elif box == 3:
                box_nr = 3
            elif box == 4:
                box_nr = 1
        elif pos_name == "B58":
            if box == 1:
                box_nr = 5
            elif box == 2:
                box_nr = 7
            elif box == 3:
                box_nr = 6
            elif box == 4:
                box_nr = 8
        elif pos_name == "B912":
            if box == 1:
                box_nr = 11
            elif box == 2:
                box_nr = 9
            elif box == 3:
                box_nr = 12
            elif box == 4:
                box_nr = 10
        elif pos_name == "B1316":
            if box == 1:
                box_nr = 14
            elif box == 2:
                box_nr = 16
            elif box == 3:
                box_nr = 13
            elif box == 4:
                box_nr = 15
        
        frames_10 = int(frames_20/2)
        end_mir = mirror_drop + frames_10
        
        if box_open + frames_20 < mirror_drop:
            end_OF = box_open+frames_20
            OF_df = box_df[(box_df.Frame > box_open) & (box_df.Frame <= end_OF)]
            OF_df = OF_df.reset_index(drop=True)
            det_OF = str(len(OF_df)) + "/" + str(frames_20)
            if lids == True:
                red_pt_OF = len(OF_df[OF_df.Lid_pt == "Red"])
                blue_pt_OF = len(OF_df[OF_df.Lid_pt == "Blue"])
                red_box_OF = len(OF_df[OF_df.Lid_box == "Red"])
                blue_box_OF = len(OF_df[OF_df.Lid_box == "Blue"])
            else:
                red_pt_OF = "NA"
                blue_pt_OF = "NA"
                red_box_OF = "NA"
                blue_box_OF = "NA"
            
            short = False
        else:
            end_OF = mirror_drop
            OF_df = box_df[(box_df.Frame > box_open) & (box_df.Frame <= mirror_drop)]
            OF_df = OF_df.reset_index(drop=True)
            frames = mirror_drop - box_open
            det_OF = str(len(OF_df)) + "/" + str(frames)
            if lids == True:
                red_pt_OF = len(OF_df[OF_df.Lid_pt == "Red"])
                blue_pt_OF = len(OF_df[OF_df.Lid_pt == "Blue"])
                red_box_OF = len(OF_df[OF_df.Lid_box == "Red"])
                blue_box_OF = len(OF_df[OF_df.Lid_box == "Blue"])
            else:
                red_pt_OF = "NA"
                blue_pt_OF = "NA"
                red_box_OF = "NA"
                blue_box_OF = "NA"
            short = str(frames) + "/" + str(frames_20)
        
        x_list = OF_df.x_coord.tolist()
        y_list = OF_df.y_coord.tolist()
        frame_list = box_df.Frame.to_list()
        
        dist_list = []
        max_frames = 0
        for index, row in OF_df.iterrows():
            if index > 0:
                dist_temp = dist_func([x_list[index-1],y_list[index-1]],[x_list[index],y_list[index]])
                
                dist_list.append(dist_temp)
                diff_frames = frame_list[index] - frame_list[index - 1]
                if diff_frames > max_frames:
                    max_frames = diff_frames
        dist_pix = sum(dist_list)
        dist = dist_pix/pix_to_cm
        
        frames_mirror = len(box_df[(box_df.Frame > mirror_drop) & (box_df.Frame <= end_mir)])
        detect_mirror = len(box_df[(box_df.Mirror == "Yes") & (box_df.Frame > mirror_drop) & (box_df.Frame <= end_mir)])
        frames_detect = len(box_df)
        det_mir = str(frames_mirror) + "/" + str(frames_10)
        if lids == True:
            red_pt_e = len(box_df[(box_df.Lid_pt == "Red") & (box_df.Frame < box_open)])
            blue_pt_e = len(box_df[(box_df.Lid_pt == "Blue") & (box_df.Frame < box_open)])
            red_box_e = len(box_df[(box_df.Lid_box == "Red") & (box_df.Frame < box_open)])
            blue_box_e = len(box_df[(box_df.Lid_box == "Blue") & (box_df.Frame < box_open)])
        else:
            red_pt_e = "NA"
            blue_pt_e = "NA"
            red_box_e = "NA"
            blue_box_e = "NA"
        first_frame = box_df.Frame.tolist()[0]
        
        if door_open == "NA":
            full_emerge = "NA"
            emerged = "NA"
        else:
            emerge_end = int(door_open) + frames_15
            if emerge_end < box_open:
                full_emerge = True
            else:
                full_emerge = False
            
            if first_frame < int(door_open):
                emerged = "Before"
            elif first_frame >= int(door_open) and first_frame < emerge_end:
                emerged = True
            else:
                emerged = False
        
        time_mirror = int(detect_mirror)/fps
        if lids == True:
            r_pt_e_time = int(red_pt_e)/fps
            b_pt_e_time = int(blue_pt_e)/fps
            r_pt_OF_time = int(red_pt_OF)/fps
            b_pt_OF_time = int(blue_pt_OF)/fps
            r_box_e_time = int(red_box_e)/fps
            b_box_e_time = int(blue_box_e)/fps
            r_box_OF_time = int(red_box_OF)/fps
            b_box_OF_time = int(blue_box_OF)/fps
        else:
            r_pt_e_time = "NA"
            b_pt_e_time = "NA"
            r_pt_OF_time = "NA"
            b_pt_OF_time = "NA"
            r_box_e_time = "NA"
            b_box_e_time = "NA"
            r_box_OF_time = "NA"
            b_box_OF_time = "NA"
        
        temp_row = pd.DataFrame({"Video":[vid],"Vid_frames":[vid_frames],"Vid_width":[width],"fps":[fps],"Box":[box_nr],
                                 "Detected_frames":[frames_detect],"Door_open":[door_open],"First_frame":[first_frame],
                                 "Full_emergence":[full_emerge],"Emerged":[emerged],"Distance":[dist],
                                 "Shorter":[short],"Detected_frames_OF":[det_OF],"Longest_no_detect_OF":[max_frames],
                                 "Frames_mirror":[detect_mirror],"Time_mirror":[time_mirror],"Detected_frames_mir":[det_mir],
                                 "Frames_red_emerge_pt":[red_pt_e],"Time_red_emerge_pt":[r_pt_e_time],
                                 "Frames_red_OF_pt":[red_pt_OF],"Time_red_OF_pt":[r_pt_OF_time],
                                 "Frames_red_emerge_box":[red_box_e],"Time_red_emerge_box":[r_box_e_time],
                                 "Frames_red_OF_box":[red_box_OF],"Time_red_OF_box":[r_box_OF_time],
                                 "Frames_blue_emerge_pt":[blue_pt_e],"Time_blue_emerge_pt":[b_pt_e_time],
                                 "Frames_blue_OF_pt":[blue_pt_OF],"Time_blue_OF_pt":[b_pt_OF_time],
                                 "Frames_blue_emerge_box":[blue_box_e],"Time_blue_emerge_box":[b_box_e_time],
                                 "Frames_blue_OF_box":[blue_box_OF],"Time_blue_OF_box":[b_box_OF_time]})
        sum_df = pd.concat([sum_df, temp_row], ignore_index = True)
    if os.path.exists("OF_mirror/Sum_OF_mir.csv"):
        sum_df.to_csv("OF_mirror/Sum_OF_mir.csv", index=False, header=False, mode="a")
    else:
        sum_df.to_csv("OF_mirror/Sum_OF_mir.csv", index=False)

print("Done")